In [2]:
# pandas
import pandas as pd
from pandas import Series,DataFrame

# numpy, matplotlib, seaborn
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [58]:
# import dataframes
patient_df = pd.read_csv("../raw/trainingSet/training_SyncPatient.csv", sep=',')
transcript_df = pd.read_csv("../raw/trainingSet/training_SyncTranscript.csv", sep=',')

In [59]:
# some weights values are 0, which dose not make sense, we convert them into NaN value and delete them later
transcript_df['Weight'] = transcript_df['Weight'].apply(lambda x: None if x == 0 else x)

In [73]:
def incorpriate(my_df, cols):
    """
    generates a new datafram indexed by "PatientGuid" with chosen columns
    
    """
    guid = transcript_df.groupby('PatientGuid')['PatientGuid'].apply(list)
    for col in cols:
        patients = transcript_df.groupby('PatientGuid')[col].apply(list)
        
        for index in range(len(patients)):
            lst = []
            for i in range(len(patients[index])):
                if pd.notnull(patients[index][i]):
                    lst.append(patients[index][i])
            if not lst:
                patients[index] = None
            else:
                patients[index] = sum(lst)/len(lst)
                
        new_df = patients.to_frame()
        new_df['PatientGuid'] = guid.apply(lambda x: x[0])
        
        my_df = my_df.merge(new_df, on='PatientGuid')
    return my_df

In [74]:
new_df = incorpriate(patient_df, ['Height', 'Weight', 'SystolicBP', 'DiastolicBP', 'RespiratoryRate', 'Temperature'])

In [80]:
new_df.to_csv('Processed_Transcript.csv', index=False)